In [1]:
from ppclass import pp
import apss_lib
import numpy as np
#%matplotlib inline

2020-04-15 10:51:26.075120


In [2]:
case = "25M_LS120_W10"
case = "25M_LS120_W20"
case = "25M_LS300_W10"
case = "25M_LS300_W20"
case = "25M_LS30_W10"
case = "25M_LS30_W20"
case = "25M_LS0_W10"
case = "25M_LS0_W20"

In [3]:
!mkdir {case}
!mkdir {case}/pdf_sol_to_sol
!mkdir {case}/txt_sol_to_sol
!mkdir {case}/txt_per_sol
!mkdir {case}/pdf_stats

In [4]:
nn = 24 #8 #5
dim = 24 #48

xxtab = np.random.rand(nn)*dim
yytab = np.random.rand(nn)*dim

print xxtab
print yytab

#dim=5 #20
#xxtab = np.arange(dim)
#yytab = np.arange(dim)

[20.76473968 10.01977595  1.5488538  16.64160036 19.11850027  8.29402999
 23.19286234  5.8819237   6.48400432 14.66723772 18.52279233  1.09920875
 17.25324798  2.52254966  5.13112302 14.51927441 17.24893931  9.11376155
 14.06301951 22.42029354  5.09916954 22.93124199 20.12274207  3.35457979]
[20.04983225  5.0752525  13.87787159  6.62390763 21.20360637 16.04548694
 11.41878805 22.61325039 17.69241462 11.55575276  5.40525199  8.39282997
  4.54907278 14.61728498  2.78666879  1.05803572  9.3068012   5.93958063
 16.76558778 13.65131103  5.33658943  9.60092466 22.26688553  3.01930697]


In [5]:



sol = 9000
soltab = []


for xx in xxtab:
    xx = int(xx)
    print xx
    
    for yy in yytab:
        yy = int(yy)

        #print xx,yy
         
        zefile = "PSFC_LMD_LES_"+case
        ff = pp(file=zefile+"_every20.nc",var="PSFC",x=xx,y=yy,z=0).getf()

        ltstpp = 7. + np.arange(len(ff))/3700.
        utcpp = np.empty_like(ltstpp,dtype="str")
        for iii in range(len(utcpp)):
            utcpp[iii] = str(iii)
        
        #ltstpp = [str(ltst) for ltst in ltstpp]
        #utcpp = ltstpp
        
        
        dpp = ff - np.mean(ff)
        
        #print sol, dpp,utcpp,ltstpp
        
        indices, ddcount, drop, search = apss_lib.studypressure(sol,dpp,"",utcpp,ltstpp,\
                                                                zefold=case,\
                                                                ltnum=True,isplot=False)
        
        soltab = np.append(soltab,sol)
        sol = sol+1        


        
#import ppplot
#ppplot.plot1d(dpp)
#ppplot.show()


20
10
1
16
19
8
23
5
6
14
18
1
17
2
5
14
17
9
14
22
5
22
20
3


In [6]:
#nn = 25
#nn = 5
#nn = 10
#nn = 20
#nn = 48

In [7]:
#sol = 0
#
#
#
#for xx in range(nn):
#    print xx
#    for yy in range(nn):
#        #print xx,yy
#              
#        ff = pp(file="PSFC_LMD_LES25m_every10.nc",var="PSFC",x=xx,y=yy,z=0).getf()
#
#        ltstpp = 8. + np.arange(len(ff))/3700.
#        utcpp = np.empty_like(ltstpp,dtype="str")
#        for iii in range(len(utcpp)):
#            utcpp[iii] = str(iii)
#        dpp = ff - np.mean(ff)        
#
#        indices, ddcount, drop, search = apss_lib.studypressure(sol,dpp,"",utcpp,ltstpp,ltnum=True,isplot=False)
#        sol = sol+1        
#
#lastsol = sol
#        
##import ppplot
##ppplot.plot1d(dpp)
##ppplot.show()

In [8]:
### create event structured arrays
dtype = np.dtype([('drop',float), ('ltst',float), ('sol',int), ('utc','S23'), ('ratio',float)])
events = np.array([],dtype=dtype)

#np.arange(sol)

#print soltab

for iii in np.arange(len(soltab)):
    
    #print soltab[iii]
      
    #fifi, output = apss_lib.namefile(iii,"")
    fifi, output = apss_lib.namefile(soltab[iii],"")
    
    #print output
    
    ddfile = "./"+case+"/txt_per_sol/"+output+"DD_3.txt"
    
    ltst = np.loadtxt(ddfile,unpack=True,dtype=str,usecols=(2))   
    drop = np.loadtxt(ddfile,unpack=True,usecols=(1))
    utc = np.loadtxt(ddfile,unpack=True,usecols=(3),dtype='S23')
       
    nnn = drop.size
    if nnn >= 1:
        for n in np.arange(nnn):
            element = np.array((drop[n],ltst[n],iii,utc[n],1),dtype=dtype)
            events = np.append(events,element)
    else:
        pass
        #print drop,ltst,soltab[iii],utc
        #events = np.append(events,np.array((drop,ltst,soltab[iii],utc,1),dtype=dtype))

print events.size




14464


In [9]:
### ordered file
oevents = np.sort(events, order='drop')
dafile = open("./"+case+"/alldrop_ordered.txt","w")
dafile.write('%6s ; %6s ; %3s ; %23s ; %5s\n' % ("_DROP_","_LTST_","SOL","YYYY-MM-DDTHH:MM:SS.sss","RATIO"))
for ooo in oevents:
    dafile.write('%6.3f ; %6.3f ; %03d ; %s ; %5.3f\n' % (ooo['drop'],ooo['ltst'],ooo['sol'],ooo['utc'],ooo['ratio']))
dafile.close()

In [10]:
print "MIN,MAX LTST",np.min(events['ltst']),np.max(events['ltst'])
print "MIN,MAX DROP",np.min(events['drop']),np.max(events['drop'])  

################# LT HISTOGRAM
limdrop = 0.5
limdrop = 0.3
#limdrop = 2.0
nbins = 18
import matplotlib.pyplot as mpl
import ppplot
ppplot.changefont(18)
mpl.figure(figsize=(12,6))
##############################################
## drops stronger than limdrop
## + complete sols
idx = (events['drop'] < -limdrop)*(events['ratio'] > 0.99)
w = np.where(idx)
what_hist = events['ltst'][w]
##############################################
ppplot.figuref(x=16,y=6)
dabins = np.linspace(8,17,nbins+1)
#dabins = 'auto'
n, bins, patches = mpl.hist(what_hist,bins=dabins,density=True)
foo = mpl.ylabel("Probability density")
foo = mpl.xlabel("Mars LTST (hours)")
foo = mpl.xticks(bins)
foo = mpl.title("%i pressure drops stronger than %.1f Pa" % (what_hist.size, limdrop))
foo = mpl.xlim(xmin=8,xmax=17)

print "MEAN = ",np.mean(what_hist)
print "STD = ",np.std(what_hist)
#from scipy.stats import norm
#f_x = norm.pdf(bins, np.mean(what_hist), np.std(what_hist))
#x = bins
#mpl.plot(x,f_x)
####
##from scipy.stats import poisson
##X = np.arange(6,19)
##mpl.plot( X, poisson.pmf(X,np.mean(what_hist)), 'r-' )
####
ppplot.save(mode="pdf",filename="ltst_hist",folder='./'+case+'/pdf_stats/')

MIN,MAX LTST 8.927297297297297 16.285405405405406
MIN,MAX DROP -5.384 -0.295
MEAN =  12.85016777738364
STD =  1.3238774065366026


In [11]:
############################
limdrop = 0.5
limdrop = 0.3
#limdrop = 0.4
#limdrop = 1
############################

dafile = open("allDD_1.txt","w")
for drop in events['drop']:
  dafile.write('%5.3f ; %5.3f ; %5.3f ; %5.3f ; %5.3f ; %5.3f\n' % (10,10,-drop,10,10,10))
dafile.close()

bintab = [7,12,30,50]
bintab = [12,30,50]

from ddhist import histodd
for nb in bintab:
    histodd("allDD",drop=True,limrest=None,limtime=None,limwind=None,nbins=nb,limdrop=limdrop,\
            folder="./"+case+"/pdf_stats/")

### limrest: restrict limit (multiple of dx) -- greater equal
### limtime: use only data earlier than this local time 
### limdrop: use only data with deeper drop than value -- 0.5 to avoid false positives
### limwind: use only data with friction velocity larger than this value


[ 0.3         0.44497191  0.66        0.9789382   1.452       2.15366404
  3.1944      4.73806089  7.02768    10.42373396 15.460896   22.9322147
 34.0139712 ]
exponent 3.9497011190979423 variance % 0.06138015282261787
fit 10070 real 10064 pc   0 ipc   0
fit 3148 real 3231 pc   3 ipc   0
fit  984 real  709 pc  28 ipc   1
fit  308 real  225 pc  27 ipc   2
fit   96 real   94 pc   2 ipc   5
fit   30 real   19 pc  37 ipc  17
fit    9 real   13 pc  38 ipc  53
fit    3 real    1 pc  66 ipc 170


/home/aspiga/soft/conda/miniconda2/envs/py2/lib/python2.7/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


[  0.3          0.36742346   0.45         0.55113519   0.675
   0.82670279   1.0125       1.24005418   1.51875      1.86008127
   2.278125     2.79012191   3.4171875    4.18518287   5.12578125
   6.2777743    7.68867187   9.41666145  11.53300781  14.12499217
  17.29951172  21.18748826  25.94926758  31.78123238  38.92390137
  47.67184857  58.38585205  71.50777286  87.57877808 107.26165929
 131.36816711]
exponent 3.681444217672071 variance % 0.08844414870425979
fit 6373 real 6318 pc   1 ipc   0
fit 3700 real 3892 pc   5 ipc   0
fit 2149 real 2186 pc   2 ipc   0
fit 1248 real  970 pc  22 ipc   0
fit  724 real  437 pc  40 ipc   1
fit  421 real  228 pc  46 ipc   1
fit  244 real  153 pc  37 ipc   2
fit  142 real   48 pc  66 ipc   4
fit   82 real   58 pc  30 ipc   6
fit   48 real   35 pc  27 ipc  10
fit   28 real   13 pc  53 ipc  18
fit   16 real    8 pc  50 ipc  31
fit    9 real    9 pc   4 ipc  53
fit    5 real    0 pc 100 ipc  92
fit    3 real    1 pc  68 ipc 158
fit    2 real    0 pc 100 

In [12]:
def create_file_drop(soltab,file="test.txt",limdrop=0.0):
  dafile = open(file,"w")
  dafile.write('SOL RATIO N_D MAXdP\n')

  for dasol in soltab:
    fifi, output = apss_lib.namefile(dasol,"")
    ddfile = "./"+case+"/txt_per_sol/"+output+"DD_3.txt"
    drop = np.loadtxt(ddfile,unpack=True,usecols=(1))
    selectdrop = drop[drop <= -limdrop]
    ndrop = len(selectdrop)
    if ndrop > 0:
      extdrop = np.min(selectdrop)
    else:
      extdrop = 0.
    daind = np.where(sol == dasol)
    daratio = 1.
    
    try:
        dafile.write( '%03d %5.3f %03d %4.2f\n' % (dasol,daratio,ndrop,extdrop))
    except:
        print dasol
        print dasol,daratio,ndrop,extdrop
  dafile.close()
  return

#soltab = np.arange(lastsol)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_0p3.txt",limdrop=0.3)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_0p5.txt",limdrop=0.5)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_1p0.txt",limdrop=1.0)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_2p0.txt",limdrop=2.0)

In [13]:
## corrected variability
nx = 16 #32 #24 #16
ny = 4 #6
lim = 0.99 # to select regular (complete) sols

import numpy as np
import ppplot
ppplot.changefont(16)

stab = ["above_0p3","above_0p5","above_1p0","above_2p0"]
#stab = ["above_0p3"]

for sss in stab:

  sol, ratio, ndd, extdrop = np.loadtxt("./"+case+"/txt_sol_to_sol/solDD_"+sss+".txt",unpack=True,skiprows=1)
  fig = ppplot.figuref(x=nx,y=ny)
  pl = ppplot.plot1d(fig=fig)
  pl.xmin = sol[0]  - 1.0
  pl.xmax = sol[-1] + 1.0
  pl.nxticks = 10
  pl.ymin = 0
  pl.ymax = np.max(ndd/ratio)+1
  pl.nyticks = np.min([pl.ymax - pl.ymin + 1, 10])
  pl.fmt = '%.0f'
  pl.xlabel = "InSight sol"
  pl.ylabel = r'# pressure drops' #per hour'
  pl.linestyle = ""
  pl.marker = "s"
  pl.title = "pressure drops " + sss.replace("p",".").replace("_"," ") + " Pa"

  ##########################
  ## regular points first
  w = np.where(ratio >= lim)
  pl.f = ndd[w]/ratio[w]
  pl.x = sol[w]
  pl.color = "b"
  pl.make()

  ppplot.save(filename="drop_dtdvar_"+sss,mode="pdf",folder="./"+case+"/pdf_sol_to_sol/")

